# I. Lý thuyết

1) Transfer Learning thường được thực hiện trên hai dữ liệu nguồn (huấn luyện mô hình nguồn) và dữ liệu đích (huấn luyện mô hình đích) thế nào ?


A. Dữ liệu nguồn và dữ liệu đích có sự liên quan tới nhau. Những đặc trưng trong dữ liệu đích xuất hiện ở những dữ liệu nguồn.

B. Dữ liệu nguồn có số lượng classes lớn hơn dữ liệu đích.

C. Kích thước của dữ liệu nguồn rất nhỏ.

D. Dữ liệu đích ít liên quan tới dữ liệu nguồn.

***Đáp án: A.***

2) Khi nào thì chúng ta nên thực hiện fine tuning trên toàn bộ các layers của mô hình đích ?

A. Khi hai dữ liệu nguồn và đích có mức độ tương đồng cao và dữ liệu đích có kích thước lớn.

B. Khi hai dữ liệu nguồn và đích có mức độ tương đồng cao và dữ liệu đích có kích thước nhỏ.

C. Dữ liệu đích có kích thước nhỏ và dữ liệu mục tiêu có kích thước lớn.

D. Cả hai dữ liệu đích và mục tiêu đều có kích thước nhỏ.

***Đáp án: A.***


3) Các phương pháp augmentation nào sẽ kết hợp nội dung từ hai ảnh lẫn nhau và tạo ra một nhãn mềm (_soft label_) cho ảnh?

A. Rotation, Random Crop, MixUp

B. Bright Constrast, Color Shift, Addition Noise

C. CutMix, MixUp

D. Flip, Information Loss

***Đáp án: C.***


4) Quá trình xây dựng một mô hình AI trong dự án là một chu trình Machine Learning Cycle kế hợp giữa huấn luyện và gán nhãn dữ liệu. Để tiết kiệm chi phí gán nhãn chúng ta nên sử dụng phương pháp nào ?

A. Lấy mẫu ngẫu nhiên từ tập unlabeled dataset để thực hiện gán nhãn.

B. Sử dụng Active Learning để lựa chọn mẫu mang lại thông tin giúp cải thiện nhiều nhất cho hiệu suất mô hình.

C. Lựa chọn mô hình pretrained lớn nhất có thể.

D. Chỉ lựa các dữ liệu có thông tin rõ ràng, có thể phân biệt được bởi con người.

***Đáp án: B.***

5) Mô hình lớn thường đạt độ chính xác cao nhưng không deploy được trên các thiết bị edge device, IoT,... Trong khi mô hình nhỏ có thể deploy được nhưng thường có độ chính xác thấp. Phương pháp nào có thể giúp mô hình nhỏ cải thiện được độ chính xác ? Có thể lựa chọn nhiều đáp án.

A. Sử dụng active learning để lựa chọn các mẫu đại diện cho tổng thể để huấn luyện mô hình nhỏ.

B. Áp dụng augmentation để huấn luyện mô hình nhỏ.

C. Fine tuning các layers của mô hình lớn sang mô hình nhỏ.

D. Sử dụng knowledge distillation để chuyển giao tri thức từ mô hình lớn sang mô hình nhỏ.

***Đáp án: A, B, D.***

# II. Thực hành

6) Từ bộ dữ liệu [Dog and Cat](https://www.kaggle.com/c/dog-vs-cat-classification/data), hãy huấn luyện một mô hình large (chẳng hạn ResNet50) bằng cách fine-tuning lại các trọng số từ pretrained model của bộ dữ liệu ImageNet. Huấn luyện trên 5 epochs.

7) Hãy huấn luyện một mô hình small (chẳng hạn MobileNetV3) không sử dụng pretrained model trên 1 epochs.

8) Sử dụng mô hình large làm teacher để cải thiện mô hình small là student theo phương pháp knowledge distillation.

9) Áp dụng thêm các kĩ thuật data augmentation kết hợp ảnh khác nhãn để tạo thành nhãn mềm và huấn luyện cải thiện tiếp model student.

10) Giả định cần huấn luyện tiếp mô hình student với các dữ liệu mới chưa được gán nhãn. Hãy xây dựng một kĩ thuật lựa chọn mẫu dựa trên đánh giá uncertainty.

# 6) 
Từ bộ dữ liệu Dog and Cat, hãy huấn luyện một mô hình large (chẳng hạn ResNet50) bằng cách fine-tuning lại các trọng số từ pretrained model của bộ dữ liệu ImageNet. Huấn luyện trên 5 epochs.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import zipfile
import glob
import time
import pickle
import pprint
import math

%matplotlib inline
import matplotlib.pyplot as plt
import PIL

import numpy as np
import pandas as pd
import sklearn.model_selection

import torch
import torch.nn as nn
import torch
import torch.optim as optim
import torch.nn.functional as F

import torchvision
import torch.utils.data
from torch.utils.data import Dataset, DataLoader

import tqdm
import random
import torchvision.transforms as T
from torchvision import models, transforms

from sklearn.model_selection import train_test_split

In [ ]:
TRAIN_DIR = '/content/drive/MyDrive/Colab Notebooks/TowardDataScience/lesson5/train/train'
TEST_DIR = '/content/drive/MyDrive/Colab Notebooks/TowardDataScience/lesson5/test/test'
train_images = glob.glob(TRAIN_DIR+"/**/**.jpg")
test_images = glob.glob(TEST_DIR+"/**.jpg")

In [ ]:
dogs_list = [img for img in train_images if img.split("/")[-2] == "dogs"]
cats_list = [img for img in train_images if img.split("/")[-2] == "cats"]

print("Dogs Images: ",len(dogs_list))
print("Cats Images: ",len(cats_list))

class_to_int = {"dogs" : 0, "cats" : 1}
int_to_class = {0 : "dogs", 1 : "cats"}

Dogs Images:  10000
Cats Images:  10000


In [ ]:
from PIL import Image
class CatDogDataset(Dataset):
    def __init__(self, imgs, class_to_int, mode = "train", 
                 transforms = None):
        super().__init__()
        self.imgs = imgs
        self.class_to_int = class_to_int
        self.mode = mode
        self.transforms = transforms
    def __getitem__(self, idx):
        image_name = self.imgs[idx]
        if self.mode == "train" or self.mode == "val":
            img = Image.open(image_name)
            # img = img.resize((256, 256))
            ### Preparing class label
            label = self.class_to_int[image_name.split("/")[-2]]
            label = torch.tensor(label, dtype = torch.float32)
            ### Apply Transforms on image
            img = self.transforms(img)
            return img, label
        elif self.mode == "test":
            img = Image.open(image_name)
            # img = img.resize((256, 256))
            ### Apply Transforms on image
            img = self.transforms(img)
            return img, image_name
    def __len__(self):
        return len(self.imgs)

In [ ]:
def get_train_transform():
    return T.Compose([
        T.RandomHorizontalFlip(p=0.5), # Random flip with probability = 0.5
        T.RandomRotation(15), # Random rotation with angle <= 15
        # T.ColorJitter(brightness=.5, hue=.3), # Bright contrast
        T.Resize((256, 256)),
        T.RandomResizedCrop(224), # Random crop Image with shape 224
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)), # Normalize according to ImageNet distribution
    ])
    
def get_val_transform():
    return T.Compose([
        T.Resize((224, 224)),
        T.ToTensor(),
        T.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225))
    ])

In [ ]:
train_imgs, val_imgs = train_test_split(train_images, test_size = 0.2)

In [ ]:
train_dataset = CatDogDataset(train_imgs, class_to_int, mode = "train", 
                              transforms = get_train_transform())
val_dataset = CatDogDataset(val_imgs, class_to_int, mode = "val", 
                            transforms = get_val_transform())
test_dataset = CatDogDataset(test_images, class_to_int, mode = "test", 
                             transforms = get_val_transform())

train_data_loader = DataLoader(
    dataset = train_dataset,
    num_workers = 2,
    batch_size = 32,
    shuffle = True
)

val_data_loader = DataLoader(
    dataset = val_dataset,
    num_workers = 2,
    batch_size = 16,
    shuffle = True
)

test_data_loader = DataLoader(
    dataset = test_dataset,
    num_workers = 2,
    batch_size = 1,
    shuffle = False
)

In [ ]:
from torchvision.models import resnet50
model = resnet50(pretrained = True)

# Modifying Head - classifier
model.fc = nn.Sequential(
    nn.Linear(2048, 1, bias = True),
    nn.Sigmoid()
)

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 10, gamma = 0.33)

#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Loading model to device
model.to(device)

# No of epochs 
warm_up_epochs = 5
epochs = 15

In [ ]:
def accuracy(preds, trues):
    preds = [1 if preds[i] >= 0.5 else 0 for i in range(len(preds))]
    acc = [1 if preds[i] == trues[i] else 0 for i in range(len(preds))]
    acc = np.sum(acc) / len(preds)
    return (acc * 100)

In [ ]:
def train_one_epoch(train_data_loader, model, optimizer):
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()
    # model.to(device)
    model.train()
    
    for images, labels in train_data_loader:
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0], 1)) # [N, 1] - to match with preds shape
        
        #Reseting Gradients
        optimizer.zero_grad()
        
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = criterion(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy
        acc = accuracy(preds, labels)
        epoch_acc.append(acc)
        
        #Backward
        _loss.backward()
        optimizer.step()
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    train_logs["loss"].append(epoch_loss)
    train_logs["accuracy"].append(epoch_acc)
    train_logs["time"].append(total_time)
        
    return epoch_loss, epoch_acc, total_time

In [ ]:
def val_one_epoch(val_data_loader, model, best_val_acc, model_name):
    epoch_loss = []
    epoch_acc = []
    start_time = time.time()
    # model.to(device)
    model.eval()
    
    for images, labels in val_data_loader:
        
        #Loading images and labels to device
        images = images.to(device)
        labels = labels.to(device)
        labels = labels.reshape((labels.shape[0], 1)) # [N, 1] - to match with preds shape
        
        #Forward
        preds = model(images)
        
        #Calculating Loss
        _loss = criterion(preds, labels)
        loss = _loss.item()
        epoch_loss.append(loss)
        
        #Calculating Accuracy
        acc = accuracy(preds, labels)
        epoch_acc.append(acc)
    
    ###Overall Epoch Results
    end_time = time.time()
    total_time = end_time - start_time
    
    ###Acc and Loss
    epoch_loss = np.mean(epoch_loss)
    epoch_acc = np.mean(epoch_acc)
    
    ###Storing results to logs
    val_logs["loss"].append(epoch_loss)
    val_logs["accuracy"].append(epoch_acc)
    val_logs["time"].append(total_time)
    
    ###Saving best model
    if epoch_acc > best_val_acc:
        best_val_acc = epoch_acc
        torch.save(model.state_dict(),model_name+"_best.pth")
        
    return epoch_loss, epoch_acc, total_time, best_val_acc

In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr = 0.005)

# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Loading model to device
model.to(device)

# No of epochs 
warm_up_epochs = 5
epochs = 15

In [ ]:
best_val_acc = 0
for epoch in range(warm_up_epochs):
    ###Training
    loss, acc, _time = train_one_epoch(train_data_loader, model, optimizer)
    
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, model, best_val_acc, "resnet50")
    
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    


Training
Epoch 1
Loss : 0.6898
Acc : 55.8062
Time : 2920.5163

Validating
Epoch 1
Loss : 0.6469
Acc : 59.675
Time : 748.515

Training
Epoch 2
Loss : 0.6405
Acc : 62.4562
Time : 355.0851

Validating
Epoch 2
Loss : 0.6579
Acc : 57.975
Time : 33.6967

Training
Epoch 3
Loss : 0.621
Acc : 65.2438
Time : 355.2121

Validating
Epoch 3
Loss : 0.7015
Acc : 55.6
Time : 33.3593

Training
Epoch 4
Loss : 0.6035
Acc : 67.25
Time : 355.5995

Validating
Epoch 4
Loss : 0.6716
Acc : 62.675
Time : 33.4663

Training
Epoch 5
Loss : 0.5953
Acc : 68.075
Time : 354.8348

Validating
Epoch 5
Loss : 0.5904
Acc : 68.25
Time : 33.7083


# 7) 
Hãy huấn luyện một mô hình small (chẳng hạn MobileNetV3) không sử dụng pretrained model trên 1 epochs.

In [ ]:
from torchvision.models import mobilenet_v3_small
model = mobilenet_v3_small()

# Modifying Head - classifier
model.classifier._modules['3']  = nn.Sequential(
    nn.Linear(1024, 1, bias = True),
    nn.Sigmoid()
)

In [ ]:
# Xavier initialize
for layer in model.classifier[3]:
  if isinstance(layer, nn.modules.linear.Linear):
    nn.init.xavier_uniform_(layer.weight)

In [ ]:
#Loss Function
criterion = nn.BCELoss()

# Logs - Helpful for plotting after training finishes
train_logs = {"loss" : [], "accuracy" : [], "time" : []}
val_logs = {"loss" : [], "accuracy" : [], "time" : []}

# setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Loading model to device
model.to(device)

# No of epochs 
epochs = 1

In [ ]:
# Optimizer
learning_rate = 0.00001
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

# Learning Rate Scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 5, gamma = 0.5)

params_1x = [param for name, param in model.named_parameters()
             if name not in ["classifier.1.0.weight", "classifier.1.0.bias"]]
trainer = torch.optim.SGD([{'params': params_1x},
                            {'params': model.classifier.parameters(), 'lr': learning_rate * 10}],
                        lr=learning_rate, weight_decay=0.001)

In [ ]:
best_val_acc = 0
for epoch in range(epochs):
    ###Training
    loss, acc, _time = train_one_epoch(train_data_loader, model, optimizer)
    
    #Print Epoch Details
    print("\nTraining")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))
    
    ###Validation
    loss, acc, _time, best_val_acc = val_one_epoch(val_data_loader, model, best_val_acc, "mobilenet_v3_small")
    
    #Print Epoch Details
    print("\nValidating")
    print("Epoch {}".format(epoch+1))
    print("Loss : {}".format(round(loss, 4)))
    print("Acc : {}".format(round(acc, 4)))
    print("Time : {}".format(round(_time, 4)))


Training
Epoch 1
Loss : 0.6856
Acc : 55.225
Time : 144.2665

Validating
Epoch 1
Loss : 0.6919
Acc : 51.475
Time : 26.8806
